<a href="https://colab.research.google.com/github/dunkelweizen/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/Convolution_Neural_Networks_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [1]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Flatten

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(256,256,3))

for layer in resnet.layers:
    layer.trainable = False

x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

Using TensorFlow backend.


94773248/94765736 [==============================] - 3s 0us/step


In [0]:
import skimage.io as io

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_mountains = io.imread_collection('/content/drive/My Drive/module2-convolutional-neural-networks/data/train/mountain/*.jpg')

In [0]:
X_train = io.concatenate_images(train_mountains)

In [6]:
X_train.shape

(252, 256, 256, 3)

In [0]:
y_train = np.ones(len(X_train))

In [0]:
np.random.shuffle(X_train)

In [0]:
train_forests = io.imread_collection('/content/drive/My Drive/module2-convolutional-neural-networks/data/train/forest/*.jpg')

In [0]:
train_forests = io.concatenate_images(train_forests)
np.random.shuffle(train_forests)

In [0]:
X_train = np.concatenate((X_train, train_forests), axis=0)

In [12]:
X_train.shape

(520, 256, 256, 3)

In [13]:
520-252

268

In [0]:
length = len(train_forests)

In [0]:
y_train = np.concatenate((y_train, np.zeros(length)), axis=0)

In [16]:
y_train[:-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [0]:
val_mountains = io.imread_collection('/content/drive/My Drive/module2-convolutional-neural-networks/data/validation/mountain/*.jpg')
X_val = io.concatenate_images(val_mountains)
np.random.shuffle(X_val)
length = len(X_val)

In [0]:
y_val = np.ones(length)

In [0]:
val_forests = io.imread_collection('/content/drive/My Drive/module2-convolutional-neural-networks/data/validation/forest/*.jpg')
val_forests = io.concatenate_images(val_forests)
np.random.shuffle(val_forests)

In [0]:
X_val = np.concatenate((X_val, val_forests), axis=0)

In [0]:
length = len(val_forests)

In [0]:
y_val = np.concatenate((y_val, np.zeros(length)), axis=0)

## Instatiate Model

In [0]:

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


## Fit Model

In [24]:
model.fit(X_train, y_train, epochs=25, validation_data = (X_val, y_val), verbose=1, shuffle=True)

Epoch 1/25
17/17 [==============================] - 3s 166ms/step - loss: 0.1190 - accuracy: 0.9462 - val_loss: 0.0396 - val_accuracy: 0.9835
Epoch 2/25
17/17 [==============================] - 1s 83ms/step - loss: 0.0326 - accuracy: 0.9923 - val_loss: 0.0647 - val_accuracy: 0.9780
Epoch 3/25
17/17 [==============================] - 1s 83ms/step - loss: 0.0140 - accuracy: 0.9962 - val_loss: 0.2740 - val_accuracy: 0.9451
Epoch 4/25
17/17 [==============================] - 1s 82ms/step - loss: 0.0034 - accuracy: 0.9981 - val_loss: 0.0371 - val_accuracy: 0.9780
Epoch 5/25
17/17 [==============================] - 1s 83ms/step - loss: 0.0017 - accuracy: 0.9981 - val_loss: 0.2616 - val_accuracy: 0.9505
Epoch 6/25
17/17 [==============================] - 1s 83ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.0326 - val_accuracy: 0.9890
Epoch 7/25
17/17 [==============================] - 1s 83ms/step - loss: 4.4555e-05 - accuracy: 1.0000 - val_loss: 0.0294 - val_accuracy: 0.9890
Epoch 8/

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [0]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Conv2D, MaxPooling2D, Dropout

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [0]:
# Compile Model
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

In [0]:
from keras.callbacks.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)

In [29]:
# Fit Model
model.fit(X_train, y_train, epochs=25, verbose=1, validation_data=(X_val, y_val))

Train on 520 samples, validate on 182 samples
Epoch 1/25
520/520 [==============================] - 1s 3ms/step - loss: 3184.0591 - accuracy: 0.6038 - val_loss: 102.3258 - val_accuracy: 0.9176
Epoch 2/25
520/520 [==============================] - 1s 2ms/step - loss: 127.4929 - accuracy: 0.8942 - val_loss: 93.3991 - val_accuracy: 0.8736
Epoch 3/25
520/520 [==============================] - 1s 2ms/step - loss: 67.6725 - accuracy: 0.9077 - val_loss: 100.3031 - val_accuracy: 0.8077
Epoch 4/25
520/520 [==============================] - 1s 2ms/step - loss: 20.4825 - accuracy: 0.9365 - val_loss: 23.4903 - val_accuracy: 0.9231
Epoch 5/25
520/520 [==============================] - 1s 2ms/step - loss: 7.0397 - accuracy: 0.9596 - val_loss: 53.5885 - val_accuracy: 0.8297
Epoch 6/25
520/520 [==============================] - 1s 2ms/step - loss: 5.8349 - accuracy: 0.9558 - val_loss: 61.4594 - val_accuracy: 0.8352
Epoch 7/25
520/520 [==============================] - 1s 2ms/step - loss: 1.8474 - accu

# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
from keras.preprocessing.image import ImageDataGenerator as IDG

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language